In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import pandas as pd
from PIL import Image
import urllib
import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

In [ ]:
import zipfile
import os

zip_file_path = '/content/archive (2).zip'

In [ ]:
Datanew='/content/Datanew'
# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the target folder
    zip_ref.extractall(Datanew)

print("Extraction complete.")


Extraction complete.


In [ ]:
import os

folder_path = '/content/Datanew'
extensions_to_keep = ['.jpg', '.png', '.gif']  # Add the extensions you want to keep

# Delete files that are not inside a folder and do not have specified extensions
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)

    # Check if the file is a regular file (not a directory) in the root of the specified folder
    if os.path.isfile(file_path):
        # Check if the file has an extension and if it's not in the list of extensions to keep
        if '.' in file and not any(file.lower().endswith(ext) for ext in extensions_to_keep):
            os.remove(file_path)

print("Cleanup complete.")


Cleanup complete.


In [ ]:
# Get a list of all subdirectories in the specified path
subdirs = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]

In [ ]:
print(subdirs)

['sign_data']


In [ ]:
!ls /content/Datanew/sign_data/train

001	  012_forg  019       025_forg	032	  038_forg  045       051_forg	058	  064_forg
001_forg  013	    019_forg  026	032_forg  039	    045_forg  052	058_forg  065
002	  013_forg  020       026_forg	033	  039_forg  046       052_forg	059	  065_forg
002_forg  014	    020_forg  027	033_forg  040	    046_forg  053	059_forg  066
003	  014_forg  021       027_forg	034	  040_forg  047       053_forg	060	  066_forg
003_forg  015	    021_forg  028	034_forg  041	    047_forg  054	060_forg  067
004	  015_forg  022       028_forg	035	  041_forg  048       054_forg	061	  067_forg
004_forg  016	    022_forg  029	035_forg  042	    048_forg  055	061_forg  068
006	  016_forg  023       029_forg	036	  042_forg  049       055_forg	062	  068_forg
006_forg  017	    023_forg  030	036_forg  043	    049_forg  056	062_forg  069
009	  017_forg  024       030_forg	037	  043_forg  050       056_forg	063	  069_forg
009_forg  018	    024_forg  031	037_forg  044	    050_forg  057	063_forg
012	  018_forg  025       0

In [ ]:
data_dir = '/content/Datanew/sign_data'
train_dir = os.path.join(data_dir, 'train')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory # Import from the correct module

# Load the training data
train_dataset = image_dataset_from_directory(
    train_dir,
    subset='training',
    validation_split=0.2,
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 1649 files belonging to 128 classes.
Using 1320 files for training.


## Data Augmentation

In [ ]:
new_data_dir = '/content/Datanew/sign_data_augmented'
new_train_dir = os.path.join(new_data_dir, 'train')
os.makedirs(new_data_dir, exist_ok=True)
os.makedirs(new_train_dir, exist_ok=True)

In [ ]:
# Load the training data
train_dataset = image_dataset_from_directory(
    train_dir,
    subset='training',
    validation_split=0.2,
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 1649 files belonging to 128 classes.
Using 1320 files for training.


In [ ]:
# Define the data augmentation transformations
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)
])

In [ ]:
# Preprocess the data with data augmentation and save the augmented images
def augment_and_preprocess(image, label):
    image = tf.cast(image, tf.float32)

    # Get the subdirectory name
    subdir = os.path.dirname(train_dataset.filenames[label.numpy()[0]])

 # Create the subdirectory in the new directory
    new_subdir = os.path.join(new_train_dir, subdir) # Define new_subdir within the function
    os.makedirs(new_subdir, exist_ok=True)

    # Save the original image
    tf.keras.preprocessing.image.save_img(
        os.path.join(new_subdir, f"{os.path.basename(train_dataset.filenames[label.numpy()[0]])}.png"), image / 255.0
    )

Fine tuning on new data

In [ ]:
import os
if os.path.exists('/content/resnet50_weights.h5'):
    print("File exists")
else:
    print("File not found")

File exists
